In [2]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 239.4 MB 13.7 MB/s eta 0:00:01     |██████████████▉                 | 111.0 MB 18.6 MB/s eta 0:00:07
     |████████████████████████████████| 133 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 18.9 MB/s eta 0:00:01
     |████████████████████████████████| 25.8 MB 11.1 MB/s eta 0:00:01
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 3.5 MB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 11.1 MB 46.3 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 966 kB/s eta 0:00:011
     |████████████████████████████████| 5.5 MB 18.5 MB/s eta 0:00:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 396 kB 27.9 MB/s eta 0:00:01
     |████████████████████████████████| 

In [2]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.8 MB 5.4 MB/s eta 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     |████████████████████████████████| 64 kB 6.2 MB/s eta 0:00:011
     |████████████████████████████████| 107 kB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 249 kB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 24.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np
import requests
import zipfile
import os
import matplotlib.pyplot as plt

# Step 1: Download the dataset
dataset_url = "https://drive.google.com/uc?export=download&id=1GeUzNVqiixXHnTl8oNiQ2W3CynX_lsu2"
response = requests.get(dataset_url)
dataset_path = "dataset.zip"

# Save and extract the dataset
with open(dataset_path, "wb") as f:
    f.write(response.content)

with zipfile.ZipFile(dataset_path, "r") as zip_ref:
    zip_ref.extractall("dataset")

# Step 2: Read the dataset file
with open("dataset/text_file.txt", "r", encoding="utf-8") as f:  # Replace with your text file name
    data = f.read()

# Step 3: Clean and tokenize the text
data = data.lower().replace("\n", " ")  # Convert to lowercase and replace newlines

# Tokenizer for word indexing
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

# Step 4: Convert text into sequences of tokens
input_sequences = []
for line in data.split("."):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

# Step 5: Pad sequences to the same length
max_sequence_len = max(len(seq) for seq in input_sequences)
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Step 6: Split into predictors and label
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

# Step 7: Build the LSTM model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Step 8: Train the model
history = model.fit(X, y, epochs=10, verbose=1)

# Step 9: Plot training loss
plt.plot(history.history['loss'], label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()

# Step 10: Function to predict the next word
def predict_next_word(seed_text, tokenizer, model, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list],
                                                                maxlen=max_sequence_len - 1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            return word
    return ""

# Step 11: Test the model
seed_text = "the quick brown fox"
next_word = predict_next_word(seed_text, tokenizer, model, max_sequence_len)
print(f"Input: '{seed_text}' -> Predicted next word: '{next_word}'")


/Users/abhishekm/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


BadZipFile: File is not a zip file